In [20]:
from regime_module import *
from statsmodels.iolib.summary2 import summary_col
import csv
import os
import math

cur_wd = os.getcwd()
print(cur_wd)

backtest_directory = os.path.join(cur_wd, 'backtests')

D:\janko_machine_learning\playground


In [21]:
ff_factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', index_col = 0)
ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m')
ff_factors = ff_factors / 100.0
ff_factors[ff_factors == -999] = np.NaN
ff_factors[ff_factors == -99.9] = np.NaN
ff_factors = ff_factors.dropna(how = 'any', axis = 0)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
ff_factors_compare = ff_factors.copy()

In [22]:
q_factor = pd.read_csv('q5_factors.csv', index_col = 0)
q_factor.index = pd.to_datetime(q_factor.index, format= '%Y%m')
q_factor = q_factor / 100.0

factor_model = q_factor[['R_MKT', 'R_ME', 'R_IA', 'R_ROE', 'R_EG']]

In [23]:
os.listdir(backtest_directory)

['12_12', '24_12', '60_12', 'outputs']

In [24]:
all_subdirs = os.listdir(backtest_directory)
out_combined = pd.DataFrame()

for fol_bt in all_subdirs:
    if fol_bt != 'outputs':
        _, _, filenames = next(os.walk(os.path.join(backtest_directory, fol_bt)))
        name = "portfolio" + fol_bt +  "_analytics.csv"
        resultFile = open(os.path.join(backtest_directory, 'outputs', name),'w')
        resultFile.close()        
        out = pd.DataFrame()
        for bt in filenames:
            backtest = pd.read_csv(os.path.join(backtest_directory,fol_bt, bt), index_col = 0)
            eql_weight = backtest.mean(axis = 1)
            
            cumsum_eql_weight = eql_weight.cumsum()[1:]
            
            eql_weight.index = pd.to_datetime(eql_weight.index)
            union_index = eql_weight.index.intersection(ff_factors['Mkt-RF'].index)
            check_mkt = ff_factors['Mkt-RF'].loc[union_index]
            check_eql_weight = eql_weight.loc[union_index]
            downside = check_eql_weight.loc[check_mkt < 0.0].dropna()
            downside_mkt = check_mkt.loc[downside.index].dropna() 
            t = eql_weight.mean() / (eql_weight.std() / math.sqrt(len(eql_weight.index)))
            
            analytics = {}
            analytics['Monthly Mean return'] = str(round(eql_weight.mean() * 100,4))
            analytics['T-Stat'] = str(round(t,2))
            analytics['Mean return annualized'] = str(round(eql_weight.mean() * 12 * 100.,4))
            analytics['Monthly std return'] = str(round(eql_weight.std() * 100.,4))
            analytics['yearly window max drawdown'] =  100.0 * (cumsum_eql_weight.rolling(12).max()[12:] - cumsum_eql_weight.rolling(12).min()[12:]).max()
            analytics['monthly sharpe'] = eql_weight.mean() / eql_weight.std()
            analytics['downside mean return'] = str(round(downside.mean() * 100,4))
            analytics['market downside mean return'] = str(round(downside_mkt.mean() * 100,4))
            
            df = pd.DataFrame.from_dict(analytics, orient = 'index')
            df.columns = [bt.replace('.csv', '')]
            if out.empty:
                out = df.copy()
            else:
                out = pd.concat([out, df], axis = 1)
        out.to_csv(os.path.join(backtest_directory, 'outputs', name), mode = 'a')
        combined = out.T
        combined.loc[:, 'lookback for features'] = fol_bt.split('_')[0]
        combined.loc[:, 'lookback for ml model'] = fol_bt.split('_')[1]
        if out_combined.empty:
            out_combined = combined.copy()
        else:
            out_combined = pd.concat([out_combined, combined], axis = 0, sort = False)
            
out_combined.to_csv(os.path.join(backtest_directory, 'outputs', 'combined_analytics.csv'))


In [36]:
out_combined

,Monthly Mean return,T-Stat,Mean return annualized,Monthly std return,yearly window max drawdown,monthly sharpe,downside mean return,market downside mean return,lookback for features,lookback for ml model
anomaly_decile_AdaBoost_l,0.4083,7.16,4.899,1.4079,15.9799,0.289981,0.6328,-3.8177,12,12
anomaly_decile_AdaBoost_ls,0.1814,4.4,2.177,1.0179,11.6516,0.178222,0.3399,-3.8177,12,12
anomaly_decile_AdaBoost_s,-0.0741,-1.68,-0.8887,1.0904,10.8627,-0.0679158,0.0215,-3.8177,12,12
anomaly_decile_AdaBoost_scaleds,-0.4671,-2.48,-5.6053,4.6506,50.1369,-0.10044,0.2429,-3.8177,12,12
anomaly_decile_AdaBoost_scaled_l,0.4083,7.16,4.899,1.4079,15.9799,0.289981,0.6328,-3.8177,12,12
...,...,...,...,...,...,...,...,...,...,...
three_factor_etf_RF_scaleds,0.6497,0.22,7.7959,33.8799,343.249,0.0191752,20.2217,-3.5161,60,12
three_factor_etf_RF_scaled_l,0.37,1.64,4.4394,2.648,14.2343,0.139709,-1.9735,-3.4954,60,12
three_factor_etf_RF_scaled_ls,1.476,0.59,17.7123,29.3151,329.369,0.0503502,7.9862,-3.5757,60,12
three_factor_etf_simp_binary_ls,-0.1118,-0.69,-1.3412,1.9765,21.2297,-0.0565469,1.04,-4.2858,60,12


In [25]:
all_subdirs

['12_12', '24_12', '60_12', 'outputs']

In [26]:
out_combined.to_latex(os.path.join(backtest_directory, 'outputs', 'combined_analytics.txt'))

In [27]:
factor_model = q_factor.copy()
fol_bt = '12_12'
_, _, filenames = next(os.walk(os.path.join(backtest_directory, fol_bt)))
name = "portfolio" + fol_bt +  "_analytics.csv"
resultFile = open(os.path.join(backtest_directory, 'outputs', name),'w')
resultFile.close()        
out = pd.DataFrame()
all_alphas = pd.DataFrame()
all_p = pd.DataFrame()
all_mean = pd.DataFrame()
all_mean_mkt = pd.DataFrame()
all_downside_mean = pd.DataFrame()
all_market_downside_mean = pd.DataFrame()

for bt in filenames:
    backtest = pd.read_csv(os.path.join(backtest_directory,fol_bt, bt), index_col = 0)
    alphas = {}
    pvalues = {}
    means = {}
    mkt_means = {}
    dmeans = {}
    dmeans_mkt = {}
    
    for strgy in backtest.columns:
        try:
            port= pd.DataFrame(backtest[strgy])
            port.columns = ['strategy_' + strgy]
            port.index = pd.to_datetime(port.index)
            #1st step. run the regression of factors on the portfolio
            df = pd.concat([port, factor_model], axis = 1).dropna(axis =0 ,how = 'any')
            Y = port.loc[df.index]
            X = factor_model.loc[df.index]
            X['intercept'] = 1.0
            model = sm.OLS(Y,X,missing = 'drop')
            #robust covariance results
            results = model.fit(cov_type='HAC',cov_kwds={'maxlags':1})
            #results.params['intercept'] = round(results.params['intercept'] * 12.*100.0, 2)
            pvalues[strgy] = results.pvalues['intercept']
            alphas[strgy]= results.params['intercept']

            check_mkt = X['R_MKT']
            check_strgy = Y.copy()
            downside = check_strgy.loc[check_mkt < 0.0].dropna()
            downside_mkt = check_mkt.loc[downside.index].dropna() 
            mean_strgy = Y.mean()
            means[strgy] = mean_strgy.values[0]
            mkt_means[strgy] = check_mkt.mean()

            dmeans[strgy] = downside.mean().values[0]
            dmeans_mkt[strgy] = downside_mkt.mean()
        except:
            print(strgy)
            
    add = pd.DataFrame.from_dict(alphas, orient = 'index')
    add.columns = [bt.replace('.csv', '')]
    
    addp = pd.DataFrame.from_dict(pvalues, orient = 'index')
    addp.columns = [bt.replace('.csv', '')]
    
    addmeans = pd.DataFrame.from_dict(means, orient = 'index')
    addmeans.columns = [bt.replace('.csv', '')]
    
    addmeans_mkt = pd.DataFrame.from_dict(mkt_means, orient = 'index')
    addmeans_mkt.columns = [bt.replace('.csv', '')]    
    
    add_dmeans = pd.DataFrame.from_dict(dmeans, orient = 'index')
    add_dmeans.columns = [bt.replace('.csv', '')]
    
    add_dmeans_mkt = pd.DataFrame.from_dict(dmeans_mkt, orient = 'index')
    add_dmeans_mkt.columns = [bt.replace('.csv', '')]
    
    
    if all_alphas.empty:
        all_alphas = add.copy()
    else:
        all_alphas = pd.concat([all_alphas, add], axis = 1)
        
    if all_p.empty:
        all_p = addp.copy()
    else:
        all_p = pd.concat([all_p, addp], axis = 1)
        
    if all_mean.empty:
        all_mean = addmeans.copy()
    else:
        all_mean = pd.concat([all_mean, addmeans], axis = 1)
        
    if all_mean_mkt.empty:
        all_mean_mkt = addmeans_mkt.copy()
    else:
        all_mean_mkt = pd.concat([all_mean_mkt, addmeans_mkt], axis = 1)
        
    if all_downside_mean.empty:
        all_downside_mean = add_dmeans.copy()
    else:
        all_downside_mean = pd.concat([all_downside_mean, add_dmeans], axis = 1)
        
    if all_market_downside_mean.empty:
        all_market_downside_mean = add_dmeans_mkt.copy()
    else:
        all_market_downside_mean = pd.concat([all_market_downside_mean, add_dmeans_mkt], axis = 1)
    
print(all_alphas)
print(all_p)

portf_isq_1
portf_srev
portf_im_12
portf_r6_6
portf_sue_6
portf_cei
portf_r1n
portf_im_6
portf_isq_1
portf_srev
portf_im_12
portf_r6_6
portf_sue_6
portf_cei
portf_r1n
portf_im_6
portf_me_dtv_12
portf_me_r15a
portf_me_cei
portf_me_r6_1
portf_me_dtv_12
portf_me_r15a
portf_me_cei
portf_me_r6_1
Coal 
Coal 
R_IA
R_IA
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
XLC
MKTRF
SMB
MKTRF
SMB
              anomaly_decile_AdaBoost_l  anomaly_decile_AdaBoost_ls  \
portf_adm                     -0.000588                    0.000325   
portf_ivff_1                  -0.001225                   -0.001025   
portf_def_1                   -0.002319                   -0.001762   
portf_vhp                     -0.003706                    0.000705   
portf_p52w_6                  -0.004163                    0.000670   
...                                 ...                         ...   
XME                                 NaN                         NaN   
XL

In [28]:
mean_check = pd.DataFrame()

over_zero_df = {}
for col in all_mean.columns:
    check = all_mean[col].dropna()
    over_zero_df[col] = check.mean() * 100.0

over_zero_df = pd.DataFrame.from_dict(over_zero_df, orient = 'index')
over_zero_df.columns = ['mean']
over_zero_df.to_csv(os.path.join(backtest_directory, 'outputs', 'mean_ret.csv'))
mean_check = over_zero_df.copy()


mean_check_mkt = pd.DataFrame()

over_zero_df = {}
for col in all_mean_mkt.columns:
    check = all_mean_mkt[col].dropna()
    over_zero_df[col] = check.mean() * 100.0

over_zero_df = pd.DataFrame.from_dict(over_zero_df, orient = 'index')
over_zero_df.columns = ['mean market']
over_zero_df.to_csv(os.path.join(backtest_directory, 'outputs', 'mean_ret_mkt.csv'))
mean_check = pd.concat([mean_check, over_zero_df], axis = 1)

over_zero_df = {}
for col in all_downside_mean.columns:
    check = all_downside_mean[col].dropna()
    over_zero_df[col] = check.mean() * 100.0

over_zero_df = pd.DataFrame.from_dict(over_zero_df, orient = 'index')
over_zero_df.columns = ['downside mean']
over_zero_df.to_csv(os.path.join(backtest_directory, 'outputs', 'downside_mean.csv'))
mean_check = pd.concat([mean_check, over_zero_df], axis = 1)

over_zero_df = {}
for col in all_market_downside_mean.columns:
    check = all_market_downside_mean[col].dropna()
    over_zero_df[col] = check.mean() * 100.0

over_zero_df = pd.DataFrame.from_dict(over_zero_df, orient = 'index')
over_zero_df.columns = ['downside market mean']
over_zero_df.to_csv(os.path.join(backtest_directory, 'outputs', 'downside_mkt_mean.csv'))
mean_check = pd.concat([mean_check, over_zero_df], axis = 1)


In [29]:
mean_check

,mean,mean market,downside mean,downside market mean
anomaly_decile_AdaBoost_l,0.383405,0.636968,0.543595,-3.810712
anomaly_decile_AdaBoost_ls,0.181220,0.658727,0.335123,-3.862201
anomaly_decile_AdaBoost_s,-0.102916,0.692710,-0.114314,-3.904030
anomaly_decile_AdaBoost_scaleds,-0.643120,0.692710,-0.274160,-3.904030
anomaly_decile_AdaBoost_scaled_l,0.383405,0.636968,0.543595,-3.810712
...,...,...,...,...
three_factor_etf_RF_scaleds,1.451415,0.552978,23.746176,-4.099227
three_factor_etf_RF_scaled_l,0.174152,0.854429,-1.226290,-3.224887
three_factor_etf_RF_scaled_ls,1.817509,0.678693,8.386047,-3.766554
three_factor_etf_simp_binary_ls,0.197937,0.672550,0.556723,-3.691410


In [30]:
over_zero_df = {}
for col in all_p.columns:
    check = all_p[col].dropna()
    over_zero_df[col] = check.mean()

over_zero_df = pd.DataFrame.from_dict(over_zero_df, orient = 'index')
over_zero_df.columns = ['mean p-value']
average_p = over_zero_df.copy()
over_zero_df.to_csv(os.path.join(backtest_directory, 'outputs', 'p_mean.csv'))

In [31]:
over_zero_df = {}
for col in all_alphas.columns:
    check = all_alphas[col].dropna()
    over_zero_df[col] = round(check.mean() * 100.0,2)

over_zero_df = pd.DataFrame.from_dict(over_zero_df, orient = 'index')
over_zero_df.columns = ['q-factor alpha average']
average_alpha = over_zero_df.copy()
over_zero_df.to_csv(os.path.join(backtest_directory, 'outputs', 'alpha_mean.csv'))

In [32]:
over_zero_df = {}
for col in all_p.columns:
    check = all_p[col].dropna()
    over_zero = check[check <0.05 ].count() / len(check.index)
    over_zero_df[col] = round(over_zero * 100.0,0)

over_zero_df = pd.DataFrame.from_dict(over_zero_df, orient = 'index')
over_zero_df.columns = ['percentage p under .05']
pct_p = over_zero_df.copy()
over_zero_df.to_csv(os.path.join(backtest_directory, 'outputs', 'p_sign.csv'))

In [33]:
over_zero_df = {}
for col in all_alphas.columns:
    check = all_alphas[col].dropna()
    over_zero = check[check >0 ].count() / len(check.index)
    over_zero_df[col] = round(over_zero * 100.0,0)

over_zero_df = pd.DataFrame.from_dict(over_zero_df, orient = 'index')
over_zero_df.columns = ['percentage alpha greater zero']
pct_alpha = over_zero_df.copy()
over_zero_df.to_csv(os.path.join(backtest_directory, 'outputs', 'over_zero_alpha.csv'))

In [34]:
print("Averaged across the indivudal strategy")
avg_strgy = pd.concat([average_alpha, average_p, pct_alpha, pct_p], axis = 1)
avg_strgy = pd.concat([avg_strgy, mean_check], axis = 1)
avg_strgy.to_latex('individual_analytics.txt')
avg_strgy

Averaged across the indivudal strategy


,q-factor alpha average,mean p-value,percentage alpha greater zero,percentage p under .05,mean,mean market,downside mean,downside market mean
anomaly_decile_AdaBoost_l,0.07,0.479630,60.0,3.0,0.383405,0.636968,0.543595,-3.810712
anomaly_decile_AdaBoost_ls,0.12,0.477782,62.0,7.0,0.181220,0.658727,0.335123,-3.862201
anomaly_decile_AdaBoost_s,-0.05,0.490117,46.0,6.0,-0.102916,0.692710,-0.114314,-3.904030
anomaly_decile_AdaBoost_scaleds,-0.12,0.490084,49.0,5.0,-0.643120,0.692710,-0.274160,-3.904030
anomaly_decile_AdaBoost_scaled_l,0.07,0.479630,60.0,3.0,0.383405,0.636968,0.543595,-3.810712
...,...,...,...,...,...,...,...,...
three_factor_etf_RF_scaleds,6.00,0.491651,100.0,0.0,1.451415,0.552978,23.746176,-4.099227
three_factor_etf_RF_scaled_l,-0.00,0.696463,67.0,0.0,0.174152,0.854429,-1.226290,-3.224887
three_factor_etf_RF_scaled_ls,0.74,0.573379,33.0,0.0,1.817509,0.678693,8.386047,-3.766554
three_factor_etf_simp_binary_ls,0.22,0.301052,67.0,67.0,0.197937,0.672550,0.556723,-3.691410


In [35]:
print('For equal weighted strategy')
out_combined.to_latex('performance_analytics.txt')
out_combined

For equal weighted strategy


,Monthly Mean return,T-Stat,Mean return annualized,Monthly std return,yearly window max drawdown,monthly sharpe,downside mean return,market downside mean return,lookback for features,lookback for ml model
anomaly_decile_AdaBoost_l,0.4083,7.16,4.899,1.4079,15.9799,0.289981,0.6328,-3.8177,12,12
anomaly_decile_AdaBoost_ls,0.1814,4.4,2.177,1.0179,11.6516,0.178222,0.3399,-3.8177,12,12
anomaly_decile_AdaBoost_s,-0.0741,-1.68,-0.8887,1.0904,10.8627,-0.0679158,0.0215,-3.8177,12,12
anomaly_decile_AdaBoost_scaleds,-0.4671,-2.48,-5.6053,4.6506,50.1369,-0.10044,0.2429,-3.8177,12,12
anomaly_decile_AdaBoost_scaled_l,0.4083,7.16,4.899,1.4079,15.9799,0.289981,0.6328,-3.8177,12,12
...,...,...,...,...,...,...,...,...,...,...
three_factor_etf_RF_scaleds,0.6497,0.22,7.7959,33.8799,343.249,0.0191752,20.2217,-3.5161,60,12
three_factor_etf_RF_scaled_l,0.37,1.64,4.4394,2.648,14.2343,0.139709,-1.9735,-3.4954,60,12
three_factor_etf_RF_scaled_ls,1.476,0.59,17.7123,29.3151,329.369,0.0503502,7.9862,-3.5757,60,12
three_factor_etf_simp_binary_ls,-0.1118,-0.69,-1.3412,1.9765,21.2297,-0.0565469,1.04,-4.2858,60,12
